## Линейная регрессия и градиентный спуск

В этом задании вам необходимо реализовать функции MSE, MAE, L1 и L2 регуляризации и их производные соответственно. Сделать это необходимо в векторной форме (используйте всю мощь библиотеки numpy!). Векторизация операций еще пригодится нам в будущем при реализации нейронных сетей.

In [ ]:
'''
If you are using Google Colab, uncomment the next lines to download `loss_and_derivatives.py` and `boston_subset.json`
You can open and change downloaded `.py` files in Colab using the "Files" sidebar on the left.
'''
# !wget https://github.com/evgeniishch/ml-dist-mipt/raw/main/homeworks/hw1_linear_regression/boston_subset.json
# !wget https://github.com/evgeniishch/ml-dist-mipt/blob/main/homeworks/hw1_linear_regression/loss_and_derivatives.py

In [ ]:
# Run some setup code for this notebook.
import random
import numpy as np
import matplotlib.pyplot as plt

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
import json
with open('boston_subset.json', 'r') as iofile:
    dataset = json.load(iofile)
feature_matrix = np.array(dataset['data'])
targets = np.array(dataset['target'])

## Реализация функционалов потерь и их производных
Вам необходимо реализовать методы в файле `loss_and_derivatives.py`.
__В этом задании мы игнорируем свободный член (bias term)__, поэтому линейная модель будет иметь вид 
$$
\hat{\mathbf{y}} = XW
$$
где в матрице $X$ отсутствует фиктивный вектор из единиц.

Реализуйте функционалы потерь, регуляризации и их производные по весам $W$. 

Ячейка ниже позволяет автоматически подгружать изменения в файле `loss_and_derivatives.py` в этот ноутбук. Не забывайте сохранять изменения в файле перед тем, как переподгружать функции!

In [ ]:
# This dirty hack might help if the autoreload has failed for some reason
try:
    del LossAndDerivatives
except:
    pass

from loss_and_derivatives import LossAndDerivatives

Обратите внимание, что мы решаем задачу для общего случая и наши целевые переменные это тоже векторы. Поэтому при вычислении метрик необходимо усреднять ошибку по всем размерностям. Для лучшего понимания мы уже реализовали метод `mse()`, посмотрите как он работает.

In [ ]:
w = np.array([1., 1.])
x_n, y_n = feature_matrix, targets

Проверки ниже помогут проверить что вы заполнили методы в файле `loss_and_derivatives.py` правильно. Убедитесь, что ваше решение проходит все ассерты перед отправкой!

In [ ]:
w = np.array([1., 1.])
x_n, y_n = feature_matrix, targets

# Repeating data to make everything multi-dimentional
w = np.vstack([w[None, :] + 0.27, w[None, :] + 0.22, w[None, :] + 0.45, w[None, :] + 0.1]).T
y_n = np.hstack([y_n[:, None], 2*y_n[:, None], 3*y_n[:, None], 4*y_n[:, None]])

In [ ]:
reference_mse_derivative = np.array([
    [ 7.32890068, 12.88731311, 18.82128365, 23.97731238],
    [ 9.55674399, 17.05397661, 24.98807528, 32.01723714]
])
reference_l2_reg_derivative = np.array([
    [2.54, 2.44, 2.9 , 2.2 ],
    [2.54, 2.44, 2.9 , 2.2 ]
])

assert np.allclose(
    reference_mse_derivative,
    LossAndDerivatives.mse_derivative(x_n, y_n, w), rtol=1e-3
), 'Something wrong with MSE derivative'

assert np.allclose(
    reference_l2_reg_derivative,
    LossAndDerivatives.l2_reg_derivative(w), rtol=1e-3
), 'Something wrong with L2 reg derivative'

print(
    'MSE derivative:\n{} \n\nL2 reg derivative:\n{}'.format(
        LossAndDerivatives.mse_derivative(x_n, y_n, w),
        LossAndDerivatives.l2_reg_derivative(w))
)

In [ ]:
reference_mae_derivative = np.array([
    [0.19708867, 0.19621798, 0.19621798, 0.19572906],
    [0.25574138, 0.25524507, 0.25524507, 0.25406404]
])
reference_l1_reg_derivative = np.array([
    [1., 1., 1., 1.],
    [1., 1., 1., 1.]
])

assert np.allclose(
    reference_mae_derivative,
    LossAndDerivatives.mae_derivative(x_n, y_n, w), rtol=1e-3
), 'Something wrong with MAE derivative'

assert np.allclose(
    reference_l1_reg_derivative,
    LossAndDerivatives.l1_reg_derivative(w), rtol=1e-3
), 'Something wrong with L1 reg derivative'

print(
    'MAE derivative:\n{} \n\nL1 reg derivative:\n{}'.format(
        LossAndDerivatives.mae_derivative(x_n, y_n, w),
        LossAndDerivatives.l1_reg_derivative(w))
)

### Градиентный спуск на реальных данных
Ниже реализован алгоритм градиентного спуска.

In [ ]:
def get_w_by_grad(X, Y, w_0, loss_mode='mse', reg_mode=None, lr=0.05, n_steps=100, reg_coeff=0.05):
    if loss_mode == 'mse':
        loss_function = LossAndDerivatives.mse
        loss_derivative = LossAndDerivatives.mse_derivative
    elif loss_mode == 'mae':
        loss_function = LossAndDerivatives.mae
        loss_derivative = LossAndDerivatives.mae_derivative
    else:
        raise ValueError('Unknown loss function. Available loss functions: `mse`, `mae`')
    
    if reg_mode is None:
        reg_function = LossAndDerivatives.no_reg
        reg_derivative = LossAndDerivatives.no_reg_derivative # lambda w: np.zeros_like(w)
    elif reg_mode == 'l2':
        reg_function = LossAndDerivatives.l2_reg
        reg_derivative = LossAndDerivatives.l2_reg_derivative
    elif reg_mode == 'l1':
        reg_function = LossAndDerivatives.l1_reg
        reg_derivative = LossAndDerivatives.l1_reg_derivative
    else:
        raise ValueError('Unknown regularization mode. Available modes: `l1`, `l2`, None')
    
    
    w = w_0.copy()

    for i in range(n_steps):
        empirical_risk = loss_function(X, Y, w) + reg_coeff * reg_function(w)
        gradient = loss_derivative(X, Y, w) + reg_coeff * reg_derivative(w)
        gradient_norm = np.linalg.norm(gradient)
        if gradient_norm > 5.:
            gradient = gradient / gradient_norm * 5.
        w -= lr * gradient
        
        if i % 25 == 0:
            print('Step={}, loss={},\ngradient values={}\n'.format(i, empirical_risk, gradient))
    return w


Проверим как он работает

In [ ]:
# Initial weight matrix
w = np.ones((2,1), dtype=float)
y_n = targets[:, None] 

In [ ]:
w_grad = get_w_by_grad(x_n, y_n, w, loss_mode='mse', reg_mode='l2', n_steps=250)

### Сравнение с `sklearn`
Давайте сравним нашу реализацию с `sklearn`.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
lr = Ridge(alpha=0.05)
lr.fit(x_n, y_n)
print('sklearn linear regression implementation delivers MSE = {}'.format(np.mean((lr.predict(x_n) - y_n)**2)))

In [ ]:
plt.scatter(x_n[:, -1], y_n[:, -1])
plt.scatter(x_n[:, -1], x_n.dot(w_grad)[:, -1], color='orange', label='Handwritten linear regression', linewidth=5)
plt.scatter(x_n[:, -1], lr.predict(x_n), color='cyan', label='sklearn Ridge')
plt.legend()
plt.show()

While the solutions may look like a bit different, remember, that handwritten linear regression was unable to fit the bias term, it was equal to $0$ by default.

Хоть решения и выглядят немного по разному, не стоит забывать, что в нашей модели мы игнорировали свободный член (то есть он был равен 0).

### Отправьте решение
To submit your work you need to log into Yandex contest (link will be provided later) and upload the `loss_and_derivatives.py` file for the corresponding problem.

Для отправки решения вам необходимо зарегистрироваться в системе Яндекс.Контест по ссылке из репозитория и загрузить файл `loss_and_derivatives.py`.